In [ ]:
!nvidia-smi

Fri Jul 14 10:41:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install bitsandbytes==0.39.0
!pip install transformers@git+https://github.com/huggingface/transformers.git
!pip install peft@git+https://github.com/huggingface/peft.git
!pip install accelerate@git+https://github.com/huggingface/accelerate.git
!pip install -q einops==0.6.1
!pip install -q sentencepiece==0.1.99
!pip install -q xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 11.0 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-install-j84cr196/transformers_416aae8493e44c3d8270a0e1fef0f959
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-j84cr196/transformers_416aae8493e44c3d8270a0e1fef0f959
  Resolved https://github.com/huggingface/transformers.git to commit 91d7df58b6537d385e90578dac40204cb550f706
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.7 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.31.0.dev0-py3-none-any.whl size=7352266 sha256=ef3cbd1435200d8686f378

In [ ]:
!pip install -q "ray[serve]"
!pip install -q starlette
!pip install -q langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 100.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 10.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.8 

In [ ]:
%%writefile test.py
import torch
from transformers import pipeline,TextStreamer
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
#from transformers import  TextStreamer
# from fastapi import FastAPI
from fastapi.responses import StreamingResponse
from typing import Generator

import time
import ray
from ray import serve

from starlette.requests import Request




template = """Question: {question}

Answer: Let's think step by step."""

PROMPT = PromptTemplate(template=template, input_variables=["question"])

# ray.init(num_gpus=1)

class LocalLLM(HuggingFacePipeline):

  @classmethod
  def from_model_id(cls):
    model_name = "huggyllama/llama-7b"
    adapters_name = 'timdettmers/guanaco-7b'

    model = AutoModelForCausalLM.from_pretrained(
                                            model_name,
                                            load_in_4bit=True,
                                            torch_dtype=torch.bfloat16,
                                            device_map="auto",
                                            max_memory= {i: '24000MB' for i in range(torch.cuda.device_count())},
                                            quantization_config=BitsAndBytesConfig(
                                                load_in_4bit=True,
                                                bnb_4bit_compute_dtype=torch.bfloat16,
                                                bnb_4bit_use_double_quant=True,
                                                bnb_4bit_quant_type='nf4'
                                            ),
                                            )
    model = PeftModel.from_pretrained(model, adapters_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    streamer = TextStreamer(tokenizer)


    pipe =  pipeline(
                      "text-generation",
                      model=model,
                      tokenizer=tokenizer,
                      max_new_tokens=200,
                      do_sample = True,
                      top_k = 10,
                      num_return_sequences=1,
                      eos_token_id=tokenizer.eos_token_id,
                      streamer=streamer
                      )
    return cls(pipeline=pipe)


@serve.deployment(
    ray_actor_options={"num_gpus": 1},
    autoscaling_config={"min_replicas": 0, "max_replicas": 2},
)
class GPTModel:
    def __init__(self):
        self.llm = LocalLLM.from_model_id()
        self.chain = LLMChain(llm=self.llm, prompt=PROMPT)


    async def _run_chain(self, text: str):
      for resp in self.chain.run(text):
        yield resp


    async def __call__(self, request: Request)->StreamingResponse:
        text = request.query_params["text"]
        res = self._run_chain(text)
        return StreamingResponse(res, status_code=200, media_type="text/plain")

entrypoint= GPTModel.bind()

Overwriting test.py


In [ ]:
!pip install -q  pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,084 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launch

In [ ]:
!ngrok authtoken "NGROK_AUTH_TOKEN"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
import subprocess

ngrok_tunnel = ngrok.connect(8000)
if ngrok_tunnel:
  print('Public URL:', ngrok_tunnel.public_url)
  command = ["serve", "run", "test:entrypoint"]
  subprocess.run(command)

Public URL: https://a541-35-196-44-35.ngrok-free.app


KeyboardInterrupt: ignored